In [6]:
import pandas as pd
import json

In [ ]:
#   "hiv": {
#     "SDGHIV": "New HIV infections (per 1000 uninfected population)", 
#     "WHS2_138": "Deaths due to HIV/AIDS (per 100 000 population)", 
#     "HIV_0000000001": "Estimated number of people (all ages) living with HIV",
#     "HIV_0000000026": "Number of new HIV infections",
#     "HIV_0000000006": "Number of people dying from HIV-related causes"
#   }

In [9]:
with open("/home/alderd2/school/csci436/health-stat-map/indicators_key.json", 'r') as f:
    indicator_lookup = json.load(f)

In [11]:
indicator_lookup['SDGHIV']

'New HIV infections (per 1000 uninfected population)'

In [ ]:
def rename_sexes(label):
    if label.upper() == 'SEX_BTSX':
        return 'Both'
    elif label.upper() == 'SEX_MLE':
        return 'Male'
    elif label.upper() == 'SEX_FMLE':
        return 'Female'
    else:
        return

In [86]:
def split_on_sex(df, which_col):
    df_both = df[df[which_col] == 'Both']
    df_male = df[df[which_col] == 'Male']
    df_female = df[df[which_col] == 'Female']

    return df_both, df_male, df_female

def split_on_year(df, indicator, sex_col=None ):
    years = sorted(list(df['YEAR'].unique()))
    if sex_col is not None:
        sex = df[sex_col].unique()[0]
        for year in years:
            out_df = df[df['YEAR'] == year]
            out_df.to_csv(f"../cleaned_data/{indicator}/{sex.lower()}/{indicator}_{year}_{sex.lower()}.csv", index=False)
    else:
        for year in years:
            out_df = df[df['YEAR'] == year]
            out_df.to_csv(f"../cleaned_data/{indicator}/{indicator}_{year}.csv", index=False)



# SDGHIV
Contains both year and sex data $\rightarrow$ split csvs on both of these

csv per sex per year

In [2]:
path = "../data/SDGHIV.parquet"

In [31]:
drop_columns=[
'IndicatorCode',
'TimeDimType',
'Dim2',
'Dim2Type',
'Dim3Type',
'Dim3',
'DataSourceDim',
'DataSourceDimType',
'Comments',
'Date',
'TimeDimensionValue',
'TimeDimensionBegin',
'TimeDimensionEnd'
]

In [ ]:
df = pd.read_parquet(path)
df = df.drop(columns=drop_columns)
df = df.rename({'TimeDim': 'YEAR'}, axis=1)
df['Dim1'] = df['Dim1'].apply(rename_sexes)

In [52]:
df_b, df_m, df_f = split_on_sex(df, 'Dim1')

In [82]:
split_on_year(df_b, 'Dim1', 'SDGHIV')
split_on_year(df_m, 'Dim1', 'SDGHIV')
split_on_year(df_f, 'Dim1', 'SDGHIV')

# WHS2_138
No sex so just split on year

Some values under `Value` marked as "\<num" which seems to translate into being a NaN in the `NumericValue` column

Only 2 years worth of data

In [83]:
path = "../data/WHS2_138.parquet"

In [99]:
drop_columns=[
'IndicatorCode',
'TimeDimType',
'Dim1Type',
'Dim1',
'Dim2Type',
'Dim2',
'Dim3Type',
'Dim3',
'DataSourceDim',
'DataSourceDimType',
'Low',
'High',
'Comments',
'Date',
'TimeDimensionValue',
'TimeDimensionBegin',
'TimeDimensionEnd'
]

In [101]:
df = pd.read_parquet(path)
df = df.drop(columns=drop_columns)
df = df.rename({'TimeDim': 'YEAR'}, axis=1)

In [105]:
split_on_year(df, 'WHS2_138')

# HIV_0000000001

No sex so just split on year

Older years missing parent location data; minor issue

Lots of older data seems to only have data for regions and not specific countries

In [109]:
path = "../data/HIV_0000000001.parquet"

In [112]:
drop_columns=[
'IndicatorCode',
'TimeDimType',
'Dim1Type',
'Dim1',
'Dim2Type',
'Dim2',
'Dim3Type',
'Dim3',
'DataSourceDim',
'DataSourceDimType',
'Low',
'High',
'Comments',
'Date',
'TimeDimensionValue',
'TimeDimensionBegin',
'TimeDimensionEnd'
]

In [110]:
df = pd.read_parquet(path)

In [ ]:
for col in drop_columns:
    print(f"{col}: {df[col].isna().all()}")

IndicatorCode: False
TimeDimType: False
Dim1Type: True
Dim1: True
Dim2Type: True
Dim2: True
Dim3Type: True
Dim3: True
DataSourceDim: True
DataSourceDimType: True
Low: True
High: True
Comments: True
Date: False
TimeDimensionValue: False
TimeDimensionBegin: False
TimeDimensionEnd: False


In [122]:
df = df.drop(columns=drop_columns)
df = df.rename({'TimeDim': 'YEAR'}, axis=1)
df

,Id,SpatialDimType,SpatialDim,ParentLocationCode,ParentLocation,YEAR,Value,NumericValue
0,2379,COUNTRY,UZB,EUR,Europe,2000,No data,NaN
1,4440,COUNTRY,YEM,EMR,Eastern Mediterranean,2004,3200 [2200 - 4600],3200.0
2,8437,COUNTRY,MAR,EMR,Eastern Mediterranean,2014,19 000 [17 000 - 21 000],19000.0
3,8479,COUNTRY,GRD,AMR,Americas,2022,No data,NaN
4,8523,COUNTRY,THA,SEAR,South-East Asia,2013,670 000 [600 000 - 760 000],670000.0
...,...,...,...,...,...,...,...,...
4889,9694116,COUNTRY,MMR,SEAR,South-East Asia,2014,260 000 [230 000 - 290 000],260000.0
4890,9694835,COUNTRY,ARE,EMR,Eastern Mediterranean,2018,1100 [880 - 1300],1100.0
4891,9694889,COUNTRY,AFG,EMR,Eastern Mediterranean,2007,3100 [1600 - 7000],3100.0
4892,9695398,COUNTRY,SVN,EUR,Europe,2000,<200 [<200 - <500],200.0


In [123]:
split_on_year(df, 'HIV_0000000001')

# HIV_0000000026
No sex so just split on year

Older years missing parent location data; minor issue

Lots of older data seems to only have data for regions and not specific countries

In [124]:
path = "../data/HIV_0000000026.parquet"

In [131]:
drop_columns=[
'IndicatorCode',
'TimeDimType',
'Dim1Type',
'Dim1',
'Dim2Type',
'Dim2',
'Dim3Type',
'Dim3',
'DataSourceDim',
'DataSourceDimType',
'Comments',
'Date',
'TimeDimensionValue',
'TimeDimensionBegin',
'TimeDimensionEnd'
]

In [126]:
df = pd.read_parquet(path)

In [128]:
for col in drop_columns:
    print(f"{col}: {df[col].isna().unique()[0]}")

IndicatorCode: False
TimeDimType: False
Dim1Type: True
Dim1: True
Dim2Type: True
Dim2: True
Dim3Type: True
Dim3: True
DataSourceDim: True
DataSourceDimType: True
Low: False
High: False
Comments: True
Date: False
TimeDimensionValue: False
TimeDimensionBegin: False
TimeDimensionEnd: False


In [132]:
df = df.drop(columns=drop_columns)
df = df.rename({'TimeDim': 'YEAR'}, axis=1)
df

,Id,SpatialDimType,SpatialDim,ParentLocationCode,ParentLocation,YEAR,Value,NumericValue,Low,High
0,2395645,COUNTRY,BDI,AFR,Africa,2014,3300 [2200 - 4400],3300.0,2200.0,4400.0
1,692571,COUNTRY,MHL,WPR,Western Pacific,2008,No data,NaN,NaN,NaN
2,4177799,COUNTRY,LVA,EUR,Europe,2009,<500 [<500 - <500],500.0,500.0,500.0
3,9416751,COUNTRY,AFG,EMR,Eastern Mediterranean,2020,1400 [<500 - 6900],1400.0,500.0,6900.0
4,7748342,COUNTRY,ARM,EUR,Europe,2017,<500 [<500 - 540],500.0,500.0,540.0
...,...,...,...,...,...,...,...,...,...,...
4889,9622957,COUNTRY,LKA,SEAR,South-East Asia,2006,<500 [<500 - <500],500.0,500.0,500.0
4890,9623518,COUNTRY,TTO,AMR,Americas,2016,No data,NaN,NaN,NaN
4891,9643543,COUNTRY,CMR,AFR,Africa,2014,No data,NaN,NaN,NaN
4892,9649698,COUNTRY,DMA,AMR,Americas,2020,No data,NaN,NaN,NaN


In [133]:
split_on_year(df, 'HIV_0000000026')

# HIV_0000000006
No sex so just split on year

Older years missing parent location data; minor issue

Lots of older data seems to only have data for regions and not specific countries

In [151]:
path = "../data/HIV_0000000006.parquet"

In [152]:
drop_columns=[
'IndicatorCode',
'TimeDimType',
'Dim1Type',
'Dim1',
'Dim2Type',
'Dim2',
'Dim3Type',
'Dim3',
'DataSourceDim',
'DataSourceDimType',
'Comments',
'Date',
'TimeDimensionValue',
'TimeDimensionBegin',
'TimeDimensionEnd',
]

In [153]:
df = pd.read_parquet(path)

In [150]:
for col in drop_columns:
    print(f"{col}: {df[col].isna().all()}")

IndicatorCode: False
TimeDimType: False
Dim1Type: True
Dim1: True
Dim2Type: True
Dim2: True
Dim3Type: True
Dim3: True
DataSourceDim: True
DataSourceDimType: True
Comments: True
Date: False
TimeDimensionValue: False
TimeDimensionBegin: False
TimeDimensionEnd: False
Low: False
High: False


In [154]:
df = df.drop(columns=drop_columns)
df = df.rename({'TimeDim': 'YEAR'}, axis=1)
df

,Id,SpatialDimType,SpatialDim,ParentLocationCode,ParentLocation,YEAR,Value,NumericValue,Low,High
0,1684,COUNTRY,ZMB,AFR,Africa,2012,23 000 [20 000 - 26 000],23000.0,20000.0,26000.0
1,3508,COUNTRY,SMR,EUR,Europe,2005,No data,NaN,NaN,NaN
2,4167,COUNTRY,ATG,AMR,Americas,2000,No data,NaN,NaN,NaN
3,4203,COUNTRY,BGR,EUR,Europe,2017,<100 [<100 - <100],100.0,100.0,100.0
4,7001,COUNTRY,TZA,AFR,Africa,2020,31 000 [25 000 - 41 000],31000.0,25000.0,41000.0
...,...,...,...,...,...,...,...,...,...,...
4889,9687618,COUNTRY,URY,AMR,Americas,2013,<500 [<200 - <500],500.0,200.0,500.0
4890,9688056,COUNTRY,IRN,EMR,Eastern Mediterranean,2015,3000 [2000 - 4300],3000.0,2000.0,4300.0
4891,9691318,COUNTRY,TJK,EUR,Europe,2006,<500 [<500 - <500],500.0,500.0,500.0
4892,9691865,COUNTRY,BHR,EMR,Eastern Mediterranean,2014,No data,NaN,NaN,NaN


In [155]:
split_on_year(df, 'HIV_0000000006')